In [8]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2022, 1, 9), datetime.date(2022, 1, 8))

In [9]:
today = today - timedelta(days=2)
wkago = today - timedelta(days=8)
#wkago = date(2020, 12, 30)
today, wkago

(datetime.date(2022, 1, 7), datetime.date(2021, 12, 30))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [10]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

2897

In [11]:
sql = """
SELECT name
FROM stocks 
WHERE status IN ("B","I","O","S") 
ORDER BY status, name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'IP', 'KCE', 'NOBLE', 'PTT', 'RATCH', 'STA', 'TISCO', 'TSTH', 'TYCN', 'BCH', 'BGRIM', 'EPG', 'GLOBAL', 'IMH', 'IVL', 'KBANK', 'NER', 'RJH', 'SAT', 'SIS', 'SYNEX', 'TOP', 'TU', 'WHART', 'BGC', 'ROJNA', 'TCAP', 'TVO'"

### Get past one year data

In [12]:
one_year_ago = yesterday - timedelta(days=365)
one_year_ago

sql = """
SELECT * 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_year_ago, in_p)
print(sql)


SELECT * 
FROM price 
WHERE date >= '2021-01-08' AND name IN ('IP', 'KCE', 'NOBLE', 'PTT', 'RATCH', 'STA', 'TISCO', 'TSTH', 'TYCN', 'BCH', 'BGRIM', 'EPG', 'GLOBAL', 'IMH', 'IVL', 'KBANK', 'NER', 'RJH', 'SAT', 'SIS', 'SYNEX', 'TOP', 'TU', 'WHART', 'BGC', 'ROJNA', 'TCAP', 'TVO') 
ORDER BY name, date


In [13]:
df = pd.read_sql(sql, const)
df.shape

(6565, 7)

In [14]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

In [15]:
sql = """
SELECT name,status 
FROM stocks 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
stocks = pd.read_sql(sql, conlite)
stocks.set_index(['name'],inplace=True)
stocks

,status
name,
BCH,I
BGC,O
BGRIM,I
EPG,I
GLOBAL,I
IMH,I
IP,B
IVL,I
KBANK,I


### Create monitors from stocks

In [16]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors

,status
name,
BCH,I
BGC,O
BGRIM,I
EPG,I
GLOBAL,I
IMH,I
IP,B
IVL,I
KBANK,I


In [17]:
monitors.shape[0]

28

In [18]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)

In [19]:
sql = """
SELECT status, COUNT(*) AS items 
FROM stocks 
WHERE name IN (%s)
GROUP BY status
ORDER BY status
"""
sql = sql % in_p
grp = pd.read_sql(sql, conlite)
grp

,status,items
0,B,9
1,I,15
2,O,4


In [20]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [21]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(116, 1)

In [22]:
df_merge = pd.merge(monitors, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(116, 3)

In [23]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,status,_merge


In [24]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(88, 3)

In [25]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
IP,B,mai
KCE,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
STA,B,SET50
TISCO,B,SET50
TSTH,B,SET
TYCN,B,SET


In [26]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [27]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BGC,O
IP,B
KCE,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O
STA,B
TCAP,O


In [28]:
buy_candidates.shape[0]

13

In [29]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BCH,I
BGRIM,I
EPG,I
GLOBAL,I
IMH,I
IVL,I
KBANK,I
NER,I
RJH,I


In [30]:
sell_candidates.shape[0]

15